In [1]:
import pandas as pd
import os
import sys
#import xlrd
import boto3
import io
import zipfile
import glob
import shutil
import boto3

# suppress pandas purely educational warnings
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

sys.path.append(os.path.expanduser('../../'))
from scripts.utils.file_helpers import pull_gpkg_from_directory, upload_csv_aws, filter_counties
from scripts.utils.write_metadata import append_metadata

Function Call

In [2]:
filename_to_county = {
    'processed_alameDetOcc.csv': 'Alameda',
    'processed_alpinDetOcc.csv': 'Alpine',
    'processed_amadoDetOcc.csv': 'Amador',
    'processed_calavDetOcc.csv': 'Calaveras',
    'processed_colusDetOcc.csv': 'Colusa',
    'processed_contrDetOcc.csv': 'Contra Costa',
    'processed_delnoDetOcc.csv': 'Del Norte',
    'processed_eldorDetOcc.csv': 'El Dorado',
    'processed_fresnDetOcc.csv': 'Fresno',
    'processed_glennDetOcc.csv': 'Glenn',
    'processed_humboDetOcc.csv': 'Humboldt',
    'processed_imperDetOcc.csv': 'Imperial',
    'processed_inyoDetOcc.csv': 'Inyo',
    'processed_kernDetOcc.csv': 'Kern',
    'processed_kingsDetOcc.csv': 'Kings',
    'processed_laDetOcc.csv': 'Los Angeles',
    'processed_lakeDetOcc.csv': 'Lake',
    'processed_lassenDetOcc.csv': 'Lassen',
    'processed_maderDetOcc.csv': 'Madera',
    'processed_marinDetOcc.csv': 'Marin',
    'processed_maripDetOcc.csv': 'Mariposa',
    'processed_mendoDetOcc.csv': 'Mendocino',
    'processed_merceDetOcc.csv': 'Merced',
    'processed_modocDetOcc.csv': 'Modoc',
    'processed_monoDetOcc.csv': 'Mono',
    'processed_monteDetOcc.csv': 'Monterey',
    'processed_napaDetOcc.csv': 'Napa',
    'processed_nevadDetOcc.csv': 'Nevada',
    'processed_oranDetOcc.csv': 'Orange',
    'processed_placeDetOcc.csv': 'Placer',
    'processed_plumaDetOcc.csv': 'Plumas',
    'processed_riveDetOcc.csv': 'Riverside',
    'processed_sacDetOcc.csv': 'Sacramento',
    'processed_sanbeDetOcc.csv': 'San Benito',
    'processed_sanbrDetOcc.csv': 'San Bernardino',
    'processed_sandiDetOcc.csv': 'San Diego',
    'processed_sanfrDetOcc.csv': 'San Francisco',
    'processed_sanjoDetOcc.csv': 'San Joaquin',
    'processed_sanluDetOcc.csv': 'San Luis Obispo',
    'processed_sanmaDetOcc.csv': 'San Mateo',
    'processed_santbDetOcc.csv': 'Santa Barbara',
    'processed_santcDetOcc.csv': 'Santa Clara',
    'processed_scruzDetOcc.csv': 'Santa Cruz',
    'processed_shastDetOcc.csv': 'Shasta',
    'processed_sierrDetOcc.csv': 'Sierra',
    'processed_siskiDetOcc.csv': 'Siskiyou',
    'processed_solanDetOcc.csv': 'Solano',
    'processed_sonomDetOcc.csv': 'Sonoma',
    'processed_staniDetOcc.csv': 'Stanislaus',
    'processed_sutteDetOcc.csv': 'Sutter',
    'processed_tehamDetOcc.csv': 'Tehama',
    'processed_triniDetOcc.csv': 'Trinity',
    'processed_tularDetOcc.csv': 'Tulare',
    'processed_tuoluDetOcc.csv': 'Tuolumne',
    'processed_ventuDetOcc.csv': 'Ventura',
    'processed_yoloDetOcc.csv': 'Yolo',
    'processed_yubaDetOcc.csv': 'Yuba'
}
def pull_csv_or_xls_from_directory(bucket_name, directory, search_zipped=True):
    """
    Pulls CSV or XLS files from a specified directory in an S3 bucket.
    
    Parameters:
    - bucket_name (str): The name of the S3 bucket.
    - directory (str): The directory within the bucket to search for CSV or XLS files.
    - search_zipped (bool): If True, search for CSV or XLS files within zip files. If False, search for CSV or XLS files directly.
    """
    # Create an S3 client
    s3 = boto3.client('s3')

    # List objects in the specified directory
    response = s3.list_objects_v2(Bucket=bucket_name, Prefix=directory)

    # Check if objects were found
    if 'Contents' in response:
        # Iterate through each object found
        for obj in response['Contents']:
            # Get the key (filename) of the object
            key = obj['Key']
            
            # Check if the object is a .zip file
            if search_zipped and key.endswith('.zip'):
                # Download the zip file into memory
                zip_object = s3.get_object(Bucket=bucket_name, Key=key)
                zip_data = io.BytesIO(zip_object['Body'].read())
                
                # Open the zip file
                with zipfile.ZipFile(zip_data, 'r') as zip_ref:
                    # Iterate through each file in the zip
                    for file_name in zip_ref.namelist():
                        # Check if the file is a .csv or .xls file
                        if file_name.endswith('.csv') or file_name.endswith('.xls'):
                            # Read the file
                            with zip_ref.open(file_name) as file:
                                # Convert the file content to pandas DataFrame
                                if file_name.endswith('.csv'):
                                    df = pd.read_csv(file)
                                elif file_name.endswith('.xls'):
                                    df = pd.read_excel(file)
                                
                                # Save the DataFrame with a similar name as the file
                                df_name = file_name.split('.')[0]  # Remove extension
                                df.to_csv(f"{df_name}.csv", index=False)
                                print(f"Saved DataFrame as '{df_name}.csv'")
                                # You can now manipulate df as needed
            elif not search_zipped and (key.endswith('.csv') or key.endswith('.xls')):
                # Directly download the CSV or XLS file
                file_object = s3.get_object(Bucket=bucket_name, Key=key)
                file_data = io.BytesIO(file_object['Body'].read())
                # Convert the file content to pandas DataFrame
                if key.endswith('.csv'):
                    df = pd.read_csv(file_data)
                elif key.endswith('.xls'):
                    df = pd.read_excel(file_data)
                # Save the DataFrame with a similar name as the file
                df_name = key.split('/')[-1].split('.')[0]  # Extract filename without extension
                df.to_csv(f"{df_name}.csv", index=False)
                print(f"Saved DataFrame as '{df_name}.csv'")
                # You can now manipulate df as needed

    else:
        print("No objects found in the specified directory.")

@append_metadata
def combined_function(input_folder, temp_folder, final_output_folder, final_combined_csv, export=False, varname=''):
    '''
    Pulls emergency responder employment data from our AWS bucket, cleans the data, and reuploads to AWS
    for future analysis and data handling. Data is sourced from: 
    https://labormarketinfo.edd.ca.gov/geography/demoaa.html

    Methods
    -------
    Data was pulled into a folder and iteratively cleaned by converting data types, removing unwanted rows,
    and isolating for emergency responder positions and values.
    Files were then merged into one single resulting csv file containing emergency employment data for each California
    county.
    
    Parameters
    ----------
    input_folder: string
        folder containing the csv employment files
    temp_folder: string
        folder to hold cleaned files before final processing
    final_output_folder: string
        folder to hold the final csv file
    final_combined_csv: string
        name of the final csv file containing the employment data  
    export: True/False boolean
        False = will not upload resulting df containing CAL CRAI emergency management grant metric to AWS
        True = will upload resulting df containing CAL CRAI emergency management grant metric to AWS

    Script
    ------
    emergency_employment_pull.ipynb

    Note:
    This function assumes users have configured the AWS CLI such that their access key / secret key pair are stored in ~/.aws/credentials.
    See https://docs.aws.amazon.com/cli/latest/userguide/getting-started-install.html for guidance.
    '''
    print('Data transformation: convert every xls file to csv file.')
    print('Data transformation: isolate relevant columns and remove unwanted rows.')
    print('Data transformation: assign each data file its corresponding county name.')
    print('Data transformation: merge all separate files together into one csv.')

    # Ensure the temp and final output folders exist
    if not os.path.exists(temp_folder):
        os.makedirs(temp_folder)
    if not os.path.exists(final_output_folder):
        os.makedirs(final_output_folder)

    # Step 1: xls_to_csv equivalent
    for filename in os.listdir(input_folder):
        if filename.endswith(".csv"):
            input_path = os.path.join(input_folder, filename)
            output_filename = os.path.splitext(filename)[0] + ".csv"
            output_path = os.path.join(temp_folder, output_filename)

            df = pd.read_csv(input_path)

            occupation_code_index = df[df.apply(lambda row: 'Occupation Code' in str(row), axis=1)].index
            if not occupation_code_index.empty:
                df = df.iloc[occupation_code_index[0]:]
                df.columns = df.iloc[0]
                df = df.iloc[1:]
                df.reset_index(drop=True, inplace=True)

                strings_to_isolate = [
                    'Police officers 3870',
                    'Firefighting and prevention workers 3740',
                    'Registered nurses 3255',
                    'Emergency medical technicians and paramedics 3401'
                ]
                df = df[df.iloc[:, 0].astype(str).str.strip().isin(strings_to_isolate)]
            else:
                print(f"Warning: 'Occupation Code' not found in {filename}. No conversion performed.")
                continue
            
            df.to_csv(output_path, index=False)

    # Step 2: process_csv_files equivalent
    for filename in os.listdir(temp_folder):
        if filename.endswith(".csv"):
            file_path = os.path.join(temp_folder, filename)
            df = pd.read_csv(file_path)

            if 'Subject' in df.columns:
                for index, row in df.iterrows():
                    if 'Total, both sexes' in str(row['Subject']):
                        df.at[index + 1, 'Subject'] = 'Occupation Total'

                df.columns.values[2] = 'Value'
                df = df.iloc[:, :3]
                df = df[~df['Subject'].str.contains('Percent')]
                df = df[~df['Subject'].str.contains('Number')]
                df = df[~df['Subject'].str.contains('Male')]
                df = df[~df['Subject'].str.contains('Female')]
                df = df[~df['Subject'].str.contains('Total, both sexes')]

                output_file_path = os.path.join(final_output_folder, f"processed_{filename}")
                df.to_csv(output_file_path, index=False)

    # Step 3: merge_data_single_csv equivalent
    combined_data = pd.DataFrame()
    for filename in os.listdir(final_output_folder):
        if filename.endswith(".csv"):
            file_path = os.path.join(final_output_folder, filename)
            df = pd.read_csv(file_path)

            county_value = filename_to_county.get(filename)
            if county_value:
                df['County'] = county_value
                combined_data = pd.concat([combined_data, df], ignore_index=True)

    combined_data.to_csv(final_combined_csv, index=False)

        # export to csv and upload to AWS
    if export == True:
        combined_data.to_csv(final_combined_csv)
        bucket_name = 'ca-climate-index'
        directory = '1_pull_data/governance/emergency_response/ca_employment_development_dept'
        export_filename = [final_combined_csv]
        upload_csv_aws(export_filename, bucket_name, directory)

    if export == False:
        print(f'{final_combined_csv} uploaded to AWS.')

    if os.path.exists(final_combined_csv[0]):
        os.remove(final_combined_csv[0])
    

In [3]:
# pull csv from aws
bucket_name = 'ca-climate-index'
aws_dir = '1_pull_data/governance/emergency_response/ca_employment_development_dept/edd_detailed_occupations_county/'

pull_csv_or_xls_from_directory(bucket_name, aws_dir, search_zipped=False)

# Define the output folder path
output_folder = 'output_folder'

# Create the output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Find all CSV files in the current directory
source_files = glob.glob('*.csv')

# Iterate through the source files and copy only CSV files to the output folder
for file in source_files:
    # Construct the destination file path
    destination_path = os.path.join(output_folder, os.path.basename(file))
    
    # Copy the file to the output folder
    shutil.copyfile(file, destination_path)
    
    # Remove the original file
    os.remove(file)

print(f"Copied and removed {len(source_files)} CSV files.")

# Define the list of variable names
var_list = [
    'governance_edd_responder_firefighter',
    'governance_edd_responder_nurse',
    'governance_edd_responder_parametics',
    'governance_edd_responder_police'
]

# Iterate through each variable name in the list and call the combined function
for varname in var_list:
    combined_function('output_folder', 'cleaned_csv_files', 'final_cleaned_data', 'ca_emergency_employment.csv', export=False, varname='test')

# Define the folders you want to remove
folders_to_remove = ['output_folder', 'cleaned_csv_files', 'final_cleaned_data']

# Remove each folder
for folder in folders_to_remove:
    if os.path.exists(folder):
        shutil.rmtree(folder)
        print(f"Removed folder: {folder}")
    else:
        print(f"Folder not found: {folder}")

Saved DataFrame as 'alameDetOcc.csv'
Saved DataFrame as 'alpinDetOcc.csv'
Saved DataFrame as 'amadoDetOcc.csv'
Saved DataFrame as 'calavDetOcc.csv'
Saved DataFrame as 'colusDetOcc.csv'
Saved DataFrame as 'contrDetOcc.csv'
Saved DataFrame as 'delnoDetOcc.csv'
Saved DataFrame as 'eldorDetOcc.csv'
Saved DataFrame as 'fresnDetOcc.csv'
Saved DataFrame as 'glennDetOcc.csv'
Saved DataFrame as 'humboDetOcc.csv'
Saved DataFrame as 'imperDetOcc.csv'
Saved DataFrame as 'inyoDetOcc.csv'
Saved DataFrame as 'kernDetOcc.csv'
Saved DataFrame as 'kingsDetOcc.csv'
Saved DataFrame as 'laDetOcc.csv'
Saved DataFrame as 'lakeDetOcc.csv'
Saved DataFrame as 'lasseDetOcc.csv'
Saved DataFrame as 'maderDetOcc.csv'
Saved DataFrame as 'marinDetOcc.csv'
Saved DataFrame as 'maripDetOcc.csv'
Saved DataFrame as 'mendoDetOcc.csv'
Saved DataFrame as 'merceDetOcc.csv'
Saved DataFrame as 'modocDetOcc.csv'
Saved DataFrame as 'monoDetOcc.csv'
Saved DataFrame as 'monteDetOcc.csv'
Saved DataFrame as 'napaDetOcc.csv'
Saved Dat